In [1]:
from TranscriptionDataset.ArrangementDataset import ArrangementDataModule
from TranscriptionModel.ArrangementModel import ArrangementModel
from pytorch_lightning import Trainer,seed_everything
import torch
import matplotlib.pyplot as plt
from pytorch_lightning.loggers import WandbLogger
import os

plt.rcParams['figure.figsize'] = [12, 8]
print(f"Cuda : {torch.cuda.is_available()}")
torch.set_float32_matmul_precision('medium')

seed_everything(42, workers=True)
# os.environ['WANDB_MODE'] = 'offline'

Global seed set to 42


Cuda : True


In [2]:
SAMPLE_RATE = 16000
dataset = "Trainsets/massive_test2.hdf5"

In [3]:
data_module = ArrangementDataModule(
    location=dataset,
    sample_rate=SAMPLE_RATE,
    val_size=0.1,
    num_workers=4,
    batch_size=16,
)

Creating OneHotEncodings: 100%|██████████| 2036/2036 [00:02<00:00, 840.74it/s]


In [4]:
model = ArrangementModel(
    tuning_size=data_module.transform.tuning_output_size,
    capo_size=data_module.transform.capo_output_size,
    arrangement_size=data_module.transform.arrangement_output_size
)

In [5]:
# train
wandb_logger = WandbLogger(
    project="ArrangementModel"
)
trainer = Trainer(
    default_root_dir="model_weights/",
    accelerator="gpu",
    devices=-1,
    max_epochs=5,
    logger=wandb_logger
)
# trainer = Trainer()
trainer.fit(model, data_module)

wandb: WARNING `resume` will be ignored since W&B syncing is set to `offline`. Starting a new run with run id z291hbt1.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name          | Type        | Params
-----------------------------------------------
0  | conv1         | Conv1d      | 5.6 K 
1  | bn1           | BatchNorm1d | 70    
2  | pool1         | MaxPool1d   | 0     
3  | conv2         | Conv1d      | 3.7 K 
4  | bn2           | BatchNorm1d | 70    
5  | pool2         | MaxPool1d   | 0     
6  | conv3         | Conv1d      | 7.4 K 
7  | bn3           | BatchNorm1d | 140   
8  | pool3         | MaxPool1d   | 0     
9  | conv4         | Conv1d      | 14.8 K
10 | bn4           | BatchNorm1d | 140   
11 | pool4         | MaxPool1d   | 0     
12 | fcArrangement | Linear      | 852   
-----------------------------------------------
32.8 K    Trainable params
0         Non-trainable params
32.8 K    Total params
0.131     Total estimated model params s

Sanity Checking: 0it [00:00, ?it/s]

C:\Users\ritwi\anaconda3\envs\music\lib\site-packages\pytorch_lightning\utilities\data.py:134: UserWarning: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
  rank_zero_warn(
C:\Users\ritwi\anaconda3\envs\music\lib\site-packages\lightning_fabric\utilities\data.py:63: UserWarning: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

C:\Users\ritwi\anaconda3\envs\music\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
